In [2]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# 数据划分方法

In [1]:
# hold-out
from sklearn.model_selection import train_test_split

# K折交叉验证
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

# K折分布保持交叉验证
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

# 时间序列划分方法
from sklearn.model_selection import TimeSeriesSplit

# booststrap 采样
from sklearn.utils import resample

In [190]:
# X = np.zeros((20, 5))
# Y = np.array([1, 2, 3, 4] * 5)
# print(X, Y)

X = np.zeros((20, 5))
Y = np.array([1]*5 + [2]*5 + [3]*5 + [4]*5)
print(X, Y)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] [1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4]


In [188]:
# 直接按照比例拆分
# train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size = 0.2)
# print(train_y, val_y)

# 按照比例 & 标签分布划分
train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size = 0.2, stratify=Y)
print(train_y, val_y)

[2 1 3 4 1 1 3 3 4 1 4 2 2 3 4 2] [3 2 4 1]


In [191]:
kf = KFold(n_splits=5)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 2 3]
Label [1 1 1 1]

[ 0  1  2  3  8  9 10 11 12 13 14 15 16 17 18 19] [4 5 6 7]
Label [1 2 2 2]

[ 0  1  2  3  4  5  6  7 12 13 14 15 16 17 18 19] [ 8  9 10 11]
Label [2 2 3 3]

[ 0  1  2  3  4  5  6  7  8  9 10 11 16 17 18 19] [12 13 14 15]
Label [3 3 3 4]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] [16 17 18 19]
Label [4 4 4 4]



In [192]:
kf = RepeatedKFold(n_splits=5, n_repeats=2)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[ 0  1  2  3  4  7  8  9 11 12 13 15 16 17 18 19] [ 5  6 10 14]
Label [2 2 3 3]

[ 1  3  4  5  6  8  9 10 11 12 13 14 16 17 18 19] [ 0  2  7 15]
Label [1 1 2 4]

[ 0  1  2  3  5  6  7  8  9 10 11 14 15 16 18 19] [ 4 12 13 17]
Label [1 3 3 4]

[ 0  2  3  4  5  6  7  8 10 11 12 13 14 15 17 19] [ 1  9 16 18]
Label [1 2 4 4]

[ 0  1  2  4  5  6  7  9 10 12 13 14 15 16 17 18] [ 3  8 11 19]
Label [1 2 3 4]

[ 0  1  2  3  4  5  7  8  9 10 11 12 15 16 17 18] [ 6 13 14 19]
Label [2 3 3 4]

[ 0  2  3  4  6  9 10 11 12 13 14 15 16 17 18 19] [1 5 7 8]
Label [1 2 2 2]

[ 1  3  4  5  6  7  8 11 12 13 14 15 16 17 18 19] [ 0  2  9 10]
Label [1 1 2 3]

[ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 19] [12 16 17 18]
Label [3 4 4 4]

[ 0  1  2  5  6  7  8  9 10 12 13 14 16 17 18 19] [ 3  4 11 15]
Label [1 1 3 4]



In [193]:
kf = StratifiedKFold(n_splits=5)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[ 1  2  3  4  6  7  8  9 11 12 13 14 16 17 18 19] [ 0  5 10 15]
Label [1 2 3 4]

[ 0  2  3  4  5  7  8  9 10 12 13 14 15 17 18 19] [ 1  6 11 16]
Label [1 2 3 4]

[ 0  1  3  4  5  6  8  9 10 11 13 14 15 16 18 19] [ 2  7 12 17]
Label [1 2 3 4]

[ 0  1  2  4  5  6  7  9 10 11 12 14 15 16 17 19] [ 3  8 13 18]
Label [1 2 3 4]

[ 0  1  2  3  5  6  7  8 10 11 12 13 15 16 17 18] [ 4  9 14 19]
Label [1 2 3 4]



In [31]:
kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[ 0  1  2  4  5  6  7  8 11 12 13 14 15 16 17 18] [ 3  9 10 19]
Label [1 2 3 4]

[ 0  1  2  3  6  7  8  9 10 12 13 14 15 17 18 19] [ 4  5 11 16]
Label [1 2 3 4]

[ 1  2  3  4  5  7  8  9 10 11 13 14 15 16 18 19] [ 0  6 12 17]
Label [1 2 3 4]

[ 0  1  3  4  5  6  7  9 10 11 12 13 16 17 18 19] [ 2  8 14 15]
Label [1 2 3 4]

[ 0  2  3  4  5  6  8  9 10 11 12 14 15 16 17 19] [ 1  7 13 18]
Label [1 2 3 4]

[ 0  1  2  4  5  6  7  8 10 11 12 14 15 16 17 19] [ 3  9 13 18]
Label [1 2 3 4]

[ 1  2  3  4  5  7  8  9 11 12 13 14 15 16 17 18] [ 0  6 10 19]
Label [1 2 3 4]

[ 0  2  3  4  5  6  7  9 10 11 12 13 15 17 18 19] [ 1  8 14 16]
Label [1 2 3 4]

[ 0  1  2  3  6  7  8  9 10 11 13 14 16 17 18 19] [ 4  5 12 15]
Label [1 2 3 4]

[ 0  1  3  4  5  6  8  9 10 12 13 14 15 16 18 19] [ 2  7 11 17]
Label [1 2 3 4]



In [194]:
kf = TimeSeriesSplit(n_splits=5)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[0 1 2 3 4] [5 6 7]
Label [2 2 2]

[0 1 2 3 4 5 6 7] [ 8  9 10]
Label [2 2 3]

[ 0  1  2  3  4  5  6  7  8  9 10] [11 12 13]
Label [3 3 3]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13] [14 15 16]
Label [3 4 4]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] [17 18 19]
Label [4 4 4]



In [39]:
train_X, train_Y = resample(X, Y, n_samples=16)
val_X, val_Y = resample(X, Y, n_samples=4)
print(train_Y, val_Y)

[4 1 3 4 2 2 4 4 1 3 3 1 3 1 2 3] [3 1 2 3]


# 提取 Two-Sigma比赛特征

In [202]:
train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')
print(train_df.shape)
print(test_df.shape)

features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price", 
                    "num_photos", "num_features", "num_description_words","created_year", 
                    "created_month", "created_day", "listing_id", "created_hour"]

# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

(49352, 15)
(74659, 14)


In [203]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

In [204]:
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(train_df["features"].head())
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
print(train_X.shape, test_X.shape)

10                                                         
10000     Doorman Elevator Fitness_Center Cats_Allowed D...
100004    Laundry_In_Building Dishwasher Hardwood_Floors...
100007                               Hardwood_Floors No_Fee
100013                                              Pre-War
Name: features, dtype: object
(49352, 217) (74659, 217)


In [205]:
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler

In [211]:
# clf = LogisticRegression()
# clf = RandomForestClassifier()
clf = LGBMClassifier()
# clf = XGBClassifier()

In [208]:
# 这写了一个bug，你能改好吗？
ss = StandardScaler()
train_X = ss.fit_transform(train_df[features_to_use])
test_X = ss.transform(test_df[features_to_use])

kf = StratifiedKFold(n_splits=5)
test_pred = None
for train_idx, test_idx, in kf.split(train_X, train_df['interest_level']):
    
    print(train_idx, test_idx)
    clf.fit(train_X[train_idx], train_y[train_idx])
    print('Val loss', log_loss(train_y[test_idx], 
                   clf.predict_proba(train_X[test_idx])))
    
    if test_pred is None:
        test_pred = clf.predict_proba(test_X)
    else:
        test_pred += clf.predict_proba(test_X)

test_pred /= 5

[ 9844  9845  9846 ... 49349 49350 49351] [    0     1     2 ... 10062 10066 10079]
Val loss 0.6996666866964564
[    0     1     2 ... 49349 49350 49351] [ 9844  9845  9846 ... 20058 20059 20083]
Val loss 0.6976671217929108
[    0     1     2 ... 49349 49350 49351] [19674 19675 19676 ... 29975 29995 29999]
Val loss 0.6949027112712386
[    0     1     2 ... 49349 49350 49351] [29513 29514 29515 ... 39683 39684 39705]
Val loss 0.6868385978392189
[    0     1     2 ... 39683 39684 39705] [39411 39413 39415 ... 49349 49350 49351]
Val loss 0.69737307995299


In [212]:
train_X = train_df[features_to_use].values
test_X = test_df[features_to_use].values

kf = StratifiedKFold(n_splits=5)
test_pred = None
for train_idx, test_idx, in kf.split(train_X, train_df['interest_level']):
    
    print(train_idx, test_idx)
    clf.fit(train_X[train_idx], train_y[train_idx])
    print('Val loss', log_loss(train_y[test_idx], 
                   clf.predict_proba(train_X[test_idx])))
    
    if test_pred is None:
        test_pred = clf.predict_proba(test_X)
    else:
        test_pred += clf.predict_proba(test_X)

test_pred /= 5

[ 9844  9845  9846 ... 49349 49350 49351] [    0     1     2 ... 10062 10066 10079]
Val loss 0.5847288585015772
[    0     1     2 ... 49349 49350 49351] [ 9844  9845  9846 ... 20058 20059 20083]
Val loss 0.5742987816612107
[    0     1     2 ... 49349 49350 49351] [19674 19675 19676 ... 29975 29995 29999]
Val loss 0.5770764063602362
[    0     1     2 ... 49349 49350 49351] [29513 29514 29515 ... 39683 39684 39705]
Val loss 0.5731740093433958
[    0     1     2 ... 39683 39684 39705] [39411 39413 39415 ... 49349 49350 49351]
Val loss 0.5823743262300808


In [218]:
# lightGBM
clf = LGBMClassifier(learning_rate=0.01, n_estimators=2000, n_jobs=2)
# clf = LGBMClassifier(n_estimators=500)

train_X = train_df[features_to_use].values
test_X = test_df[features_to_use].values

kf = StratifiedKFold(n_splits=5)
test_pred = None
for train_idx, test_idx, in kf.split(train_X, train_df['interest_level']):
    
    print(train_idx, test_idx)
    clf.fit(train_X[train_idx], train_y[train_idx], 
            eval_set=[(train_X[test_idx], train_y[test_idx]), (train_X[test_idx], train_y[test_idx])],
           verbose=20, early_stopping_rounds=50)
    print('Val loss', log_loss(train_y[test_idx], 
                   clf.predict_proba(train_X[test_idx])))
    
    if test_pred is None:
        test_pred = clf.predict_proba(test_X)
    else:
        test_pred += clf.predict_proba(test_X)

test_pred /= 5

[ 9844  9845  9846 ... 49349 49350 49351] [    0     1     2 ... 10062 10066 10079]
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_logloss: 0.749777	valid_1's multi_logloss: 0.749777
[40]	valid_0's multi_logloss: 0.722504	valid_1's multi_logloss: 0.722504
[60]	valid_0's multi_logloss: 0.702093	valid_1's multi_logloss: 0.702093
[80]	valid_0's multi_logloss: 0.686299	valid_1's multi_logloss: 0.686299
[100]	valid_0's multi_logloss: 0.673796	valid_1's multi_logloss: 0.673796
[120]	valid_0's multi_logloss: 0.663649	valid_1's multi_logloss: 0.663649
[140]	valid_0's multi_logloss: 0.655094	valid_1's multi_logloss: 0.655094
[160]	valid_0's multi_logloss: 0.648178	valid_1's multi_logloss: 0.648178
[180]	valid_0's multi_logloss: 0.642242	valid_1's multi_logloss: 0.642242
[200]	valid_0's multi_logloss: 0.637278	valid_1's multi_logloss: 0.637278
[220]	valid_0's multi_logloss: 0.632902	valid_1's multi_logloss: 0.632902
[240]	valid_0's multi_logloss: 0.62907	valid

[120]	valid_0's multi_logloss: 0.657019	valid_1's multi_logloss: 0.657019
[140]	valid_0's multi_logloss: 0.648077	valid_1's multi_logloss: 0.648077
[160]	valid_0's multi_logloss: 0.640877	valid_1's multi_logloss: 0.640877
[180]	valid_0's multi_logloss: 0.634722	valid_1's multi_logloss: 0.634722
[200]	valid_0's multi_logloss: 0.629364	valid_1's multi_logloss: 0.629364
[220]	valid_0's multi_logloss: 0.624855	valid_1's multi_logloss: 0.624855
[240]	valid_0's multi_logloss: 0.620393	valid_1's multi_logloss: 0.620393
[260]	valid_0's multi_logloss: 0.616875	valid_1's multi_logloss: 0.616875
[280]	valid_0's multi_logloss: 0.613883	valid_1's multi_logloss: 0.613883
[300]	valid_0's multi_logloss: 0.611285	valid_1's multi_logloss: 0.611285
[320]	valid_0's multi_logloss: 0.608857	valid_1's multi_logloss: 0.608857
[340]	valid_0's multi_logloss: 0.606441	valid_1's multi_logloss: 0.606441
[360]	valid_0's multi_logloss: 0.604212	valid_1's multi_logloss: 0.604212
[380]	valid_0's multi_logloss: 0.60213

[260]	valid_0's multi_logloss: 0.619195	valid_1's multi_logloss: 0.619195
[280]	valid_0's multi_logloss: 0.616262	valid_1's multi_logloss: 0.616262
[300]	valid_0's multi_logloss: 0.613447	valid_1's multi_logloss: 0.613447
[320]	valid_0's multi_logloss: 0.610771	valid_1's multi_logloss: 0.610771
[340]	valid_0's multi_logloss: 0.60851	valid_1's multi_logloss: 0.60851
[360]	valid_0's multi_logloss: 0.60624	valid_1's multi_logloss: 0.60624
[380]	valid_0's multi_logloss: 0.604125	valid_1's multi_logloss: 0.604125
[400]	valid_0's multi_logloss: 0.602058	valid_1's multi_logloss: 0.602058
[420]	valid_0's multi_logloss: 0.60031	valid_1's multi_logloss: 0.60031
[440]	valid_0's multi_logloss: 0.59883	valid_1's multi_logloss: 0.59883
[460]	valid_0's multi_logloss: 0.5974	valid_1's multi_logloss: 0.5974
[480]	valid_0's multi_logloss: 0.595966	valid_1's multi_logloss: 0.595966
[500]	valid_0's multi_logloss: 0.594764	valid_1's multi_logloss: 0.594764
[520]	valid_0's multi_logloss: 0.593569	valid_1's 

[400]	valid_0's multi_logloss: 0.597893	valid_1's multi_logloss: 0.597893
[420]	valid_0's multi_logloss: 0.596071	valid_1's multi_logloss: 0.596071
[440]	valid_0's multi_logloss: 0.594354	valid_1's multi_logloss: 0.594354
[460]	valid_0's multi_logloss: 0.592864	valid_1's multi_logloss: 0.592864
[480]	valid_0's multi_logloss: 0.591404	valid_1's multi_logloss: 0.591404
[500]	valid_0's multi_logloss: 0.590062	valid_1's multi_logloss: 0.590062
[520]	valid_0's multi_logloss: 0.588786	valid_1's multi_logloss: 0.588786
[540]	valid_0's multi_logloss: 0.587629	valid_1's multi_logloss: 0.587629
[560]	valid_0's multi_logloss: 0.586537	valid_1's multi_logloss: 0.586537
[580]	valid_0's multi_logloss: 0.58558	valid_1's multi_logloss: 0.58558
[600]	valid_0's multi_logloss: 0.584657	valid_1's multi_logloss: 0.584657
[620]	valid_0's multi_logloss: 0.583744	valid_1's multi_logloss: 0.583744
[640]	valid_0's multi_logloss: 0.582921	valid_1's multi_logloss: 0.582921
[660]	valid_0's multi_logloss: 0.582123	

[540]	valid_0's multi_logloss: 0.595648	valid_1's multi_logloss: 0.595648
[560]	valid_0's multi_logloss: 0.594647	valid_1's multi_logloss: 0.594647
[580]	valid_0's multi_logloss: 0.5936	valid_1's multi_logloss: 0.5936
[600]	valid_0's multi_logloss: 0.592576	valid_1's multi_logloss: 0.592576
[620]	valid_0's multi_logloss: 0.591617	valid_1's multi_logloss: 0.591617
[640]	valid_0's multi_logloss: 0.590759	valid_1's multi_logloss: 0.590759
[660]	valid_0's multi_logloss: 0.589956	valid_1's multi_logloss: 0.589956
[680]	valid_0's multi_logloss: 0.589147	valid_1's multi_logloss: 0.589147
[700]	valid_0's multi_logloss: 0.588488	valid_1's multi_logloss: 0.588488
[720]	valid_0's multi_logloss: 0.587833	valid_1's multi_logloss: 0.587833
[740]	valid_0's multi_logloss: 0.587177	valid_1's multi_logloss: 0.587177
[760]	valid_0's multi_logloss: 0.586608	valid_1's multi_logloss: 0.586608
[780]	valid_0's multi_logloss: 0.585953	valid_1's multi_logloss: 0.585953
[800]	valid_0's multi_logloss: 0.585268	va

In [185]:
out_df = pd.DataFrame(test_pred)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_starter2.csv", index=False)

# 参数搜索

In [216]:
LGBMClassifier?

In [217]:
from sklearn.metrics import make_scorer
def my_scorer(clf, X, y_true):
    class_labels = clf.classes_
    y_pred_proba = clf.predict_proba(X)
    return log_loss(y_true, y_pred_proba)

from sklearn.model_selection import GridSearchCV
parameters = {
    'num_leaves':( 4, 8, 16, 32), 
    'subsample':(0.75, 0.85, 0.95),
    'min_child_samples': (5, 10, 15)
}

clf = GridSearchCV(LGBMClassifier(), param_grid=parameters, n_jobs=6, scoring=my_scorer, cv=5)
clf.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=6,
             param_grid={'min_child_samples': (5, 10, 15),
                         'num_leaves':

In [153]:
from sklearn.metrics import make_scorer
def my_scorer(clf, X, y_true):
    class_labels = clf.classes_
    y_pred_proba = clf.predict_proba(X)
    return log_loss(y_true, y_pred_proba)

from sklearn.model_selection import RandomizedSearchCV
parameters = {
    'num_leaves':( 4, 8, 16, 32), 
    'subsample':[0.75, 1],
    'min_child_samples': (5, 10, 15)
}

clf = GridSearchCV(LGBMClassifier(), param_grid=parameters, 
                   n_jobs=6, scoring=my_scorer, cv=StratifiedKFold(n_splits=5))
clf.fit(train_X, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=6,
             param_grid=

# 阅读链接

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://lightgbm.readthedocs.io/en/latest/Python-API.html
- https://xgboost.readthedocs.io/en/latest//python/index.html